# Evolutionary Camouflage Versus a Learning Predator
EvoCamoVsLearningPredator.ipynb

Just a copy of Evo_Camo_vs_Static_FCD.ipynb as of 20220403


In [47]:
# Shared "communication" directory on Drive.
shared_directory = '/content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/'

# Pathname of pre-trained Keras/TensorFlow model
saved_model_directory = '/content/drive/My Drive/PredatorEye/saved_models/'
# trained_model = saved_model_directory + '20220202_1211_Find_3_Disks_complex'
# trained_model = saved_model_directory + '20220222_1747_F3D_augmented_rc4'
# trained_model = saved_model_directory + '20220227_0746_F3D2_a'
# trained_model = saved_model_directory + '20220304_1135_FCD5_a'
trained_model = saved_model_directory + '20220321_1711_FCD6_rc4'
model = []

my_prefix = "find_"
other_prefix = "camo_"

my_suffix =  ".txt"
# other_suffix = ".jpeg"
other_suffix = ".png"

fcd_image_size = 1024
fcd_disk_size = 201

import time
import PIL
from os import listdir
from os import remove
from os.path import join
from os.path import split
from os.path import isfile
from tensorflow import keras

# from PIL import Image
import numpy as np

%tensorflow_version 2.x
import tensorflow as tf
print('TensorFlow version:', tf.__version__)

from tensorflow.keras import backend as keras_backend
keras_backend.set_image_data_format('channels_last')

# Import DiskFind utilities for PredatorEye.
import sys
sys.path.append('/content/drive/My Drive/PredatorEye/shared_code/')
import DiskFind as df

TensorFlow version: 2.8.0


# Ad hoc “predator server”

In [48]:
# Top level: wait for camo_xxx.jpeg files to appear, respond with find_xxx.txt
def start_run(step = 0):
    if step == 0:
        print('Start run in', shared_directory )
        list_unexpected_files(shared_directory)
    else:
        print('Continue run at step', step, ' in', shared_directory)
    while True:
        performStep(step, shared_directory)
        step += 1

# Continue from from the last camo_xxx.jpeg file.
def restart_run():
    start_run(newest_file_from_other(shared_directory))

# Single step: wait for camo file, write response, delete previous response.
def performStep(step, directory):
    waitForReply(step, shared_directory)
    print('Write file', step)
    writeResponseFile(step, shared_directory)
    deleteMyFile(step - 1, shared_directory)

# Read image file for step, apply pre-trained model, write response file.
def writeResponseFile(step, directory):
    # Read image file and check for expected format.
    image_pathname = makeOtherPathname(step, directory)
    pixel_tensor = df.read_image_file_as_pixel_tensor(image_pathname)
    assert df.check_pixel_tensor(pixel_tensor), ('wrong file format: ' +
                                                 image_pathname)
    # Run pre-trained model on new image.
    predict = model.predict(tf.convert_to_tensor([pixel_tensor]))[0]
    # Generate response file.
    response_string = str(predict[0]) + " " + str(predict[1])
    print('response_string ' + "'" + response_string + "'")
    file = open(makeMyPathname(step, directory),"w")
    file.write(response_string)
    file.close()
    print("wrote response file", makeMyPathname(step, directory))
    ############################################################################

    # 20220408 make into utility?
    # Now "fine tune" the predator model using ground truth prey center location
    # data provided in "prey_n.txt" file.
    prey_centers = read_3_centers_from_file(step, directory)
    print('prey_centers =\n', prey_centers)
    print('predict =', predict)
    print('type(predict) =', type(predict))

    # SURELY there is a more "pythonic" way to do this (select min distance)
    min_distance = float('inf')
    nearest_center = []
    for prey_center in prey_centers:
        print('prey_center =', prey_center)
        distance = df.dist2d(prey_center, predict)
        print('distance =', distance)
        if min_distance > distance:
            min_distance = distance
            nearest_center = prey_center
        print('min_distance =', min_distance)
        print('nearest_center =', nearest_center)

    ############################################################################

# Delete the given file, usually after having written the next one.
def deleteMyFile(step, directory):
    path = makeMyPathname(step, directory)
    if isfile(path):
        remove(path)

# From pathname for file of given step number from the "other" agent.
def makeOtherPathname(step, directory):
    return directory + other_prefix + str(step) + other_suffix

# Form pathname for file of given step number from "this" agent.
def makeMyPathname(step, directory):
    return directory + my_prefix + str(step) + my_suffix

################################################################################
def makePreyPathname(step, directory):
    return directory + 'prey_' + str(step) + '.txt'
################################################################################

# Wait until other agent's file for given step appears.
def waitForReply(step, directory):
    print('start waiting for  ', makeOtherPathname(step, directory))
    start_time = time.time()
    while not isFilePresent(makeOtherPathname(step, directory)):
        time.sleep(2)  # wait 2 sec
    print('done waiting for   ', makeOtherPathname(step, directory))
    print('Elapsed time:', int(time.time() - start_time), 'seconds.')

# Like fs::exists() but for unknown reasons, that does not
# seem to work for newly created files on G Drive.
#
# TODO Why? This version works on G Drive, but it seems simply
#      calling fs::exists() should be enough.
#
def isFilePresent(file):
    result = False
    (directory, filename) = split(file)
    for i in listdir(directory):
        if i == filename:
            result = True
    return result

# Actually I guess the counterparty may have already written its first...
def list_unexpected_files(directory):
    directory_contents = listdir(directory)
    if directory_contents:
        print('Unexpected files:', directory_contents)

# Returns the step number of the newest file from "other" in given directory.
# (So if "camo_573.jpeg" is the only "other" file there, returns int 573)
def newest_file_from_other(directory):
    steps = [0]  # Default to zero in case dir is empty.
    for filename in listdir(directory):
        if other_prefix == filename[0:len(other_prefix)]:
            steps.append(int(filename.split(".")[0].split("_")[1]))
    return max(steps)

################################################################################

# TODO 20220408
# Read ground truth prey center location data provided in "prey_n.txt" file.
def read_3_centers_from_file(step, directory):
    prey_centers_file = open(makePreyPathname(step, directory),'r')
    prey_centers_string = prey_centers_file.read()
    prey_centers_file.close()
    return np.reshape(list(map(float, prey_centers_string.split())), (3, 2))

################################################################################

# Read pre-trained model

In [49]:
# Read pre-trained TensorFlow "predator vision" model.

print('Reading pre-trained model from:', trained_model)
# ad hoc workaround suggested on https://stackoverflow.com/q/66408995/1991373
#
# dependencies = {
#     'hamming_loss': tfa.metrics.HammingLoss(mode="multilabel", name="hamming_loss"),
#     'attention': attention(return_sequences=True)
# }
#
# dependencies = {
#     'valid_accuracy': ValidAccuracy
# }

# Calculates RELATIVE disk radius on the fly -- rewrite later.
def fcd_disk_radius():
    return (float(fcd_disk_size) / float(fcd_image_size)) / 2

# Given two tensors of 2d point coordinates, return a tensor of the Cartesian
# distance between corresponding points in the input tensors.
def corresponding_distances(y_true, y_pred):
    true_pos_x, true_pos_y = tf.split(y_true, num_or_size_splits=2, axis=1)
    pred_pos_x, pred_pos_y = tf.split(y_pred, num_or_size_splits=2, axis=1)
    dx = true_pos_x - pred_pos_x
    dy = true_pos_y - pred_pos_y
    distances = tf.sqrt(tf.square(dx) + tf.square(dy))
    return distances

# 20211231 copied from Find_Concpocuous_Disk
def in_disk(y_true, y_pred):
    distances = corresponding_distances(y_true, y_pred)
    # relative_disk_radius = (float(fcd_disk_size) / float(fcd_image_size)) / 2

    # From https://stackoverflow.com/a/42450565/1991373
    # Boolean tensor marking where distances are less than relative_disk_radius.
    # insides = tf.less(distances, relative_disk_radius)
    insides = tf.less(distances, fcd_disk_radius())
    map_to_zero_or_one = tf.cast(insides, tf.int32)
    return map_to_zero_or_one

dependencies = { 'in_disk': in_disk }

model = keras.models.load_model(trained_model, custom_objects=dependencies)

Reading pre-trained model from: /content/drive/My Drive/PredatorEye/saved_models/20220321_1711_FCD6_rc4


# Run test

In [52]:
# Normally start from step 0, or if an "other" file exists
# (eg 'camo_123.jpeg') then restart from that point.
restart_run()

Start run in /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/
start waiting for   /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/camo_0.png
done waiting for    /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/camo_0.png
Elapsed time: 28 seconds.
Write file 0
response_string '0.6716603 0.44930404'
wrote response file /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/find_0.txt
prey_centers =
 [[0.660561 0.41773 ]
 [0.416262 0.800781]
 [0.376737 0.50784 ]]
predict = [0.6716603  0.44930404]
type(predict) = <class 'numpy.ndarray'>
prey_center = [0.660561 0.41773 ]
distance = 0.033468116482983395
min_distance = 0.033468116482983395
nearest_center = [0.660561 0.41773 ]
prey_center = [0.416262 0.800781]
distance = 0.43447018786933156
min_distance = 0.033468116482983395
nearest_center = [0.660561 0.41773 ]
prey_center = [0.376737 0.50784 ]
distance = 0.3006762601191477
min_distance = 0.033468116482983395
nearest_center = [0.660561 0.41773 ]
start waiti

KeyboardInterrupt: ignored